# MATH 189 Final Project

by: Pranav Damal, Lucien Chen, Venz Burgos

## Objective: ...

## Thesis: ...

In [4]:
# pip install nfl_data_py
import nfl_data_py as nfl
import pandas as pd

## Top NFL QB per year 2006-2023

In [41]:
import pandas as pd
year = [2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
highestQBR = {}
for x in year:
    df = nfl.import_qbr([x])
    rank1 = df['rank'].idxmin()
    name = df.loc[rank1, 'name_display']
    highestQBR[x] = name

# highestQBR is a dict with the year and rank 1 QB
# incase this data is needed
print(highestQBR)

{2006: 'Peyton Manning', 2007: 'Tom Brady', 2008: 'Peyton Manning', 2009: 'Drew Brees', 2010: 'Tom Brady', 2011: 'Aaron Rodgers', 2012: 'Peyton Manning', 2013: 'Peyton Manning', 2014: 'Tony Romo', 2015: 'Carson Palmer', 2016: 'Matt Ryan', 2017: 'Carson Wentz', 2018: 'Patrick Mahomes', 2019: 'Lamar Jackson', 2020: 'Aaron Rodgers', 2021: 'Aaron Rodgers', 2022: 'Patrick Mahomes', 2023: 'Brock Purdy'}


## Top College QB Prospects 2006-2023

In [62]:
top_qb_prospects = {}

for year in range(2010, 2024):
    draft = nfl.import_draft_picks([year])
    firstQBs = draft[(draft['position'] == 'QB') & (draft['round'] == 1)]
    qbList = firstQBs['pfr_player_name'].tolist()
    top_qb_prospects[year] = qbList

for year, prospects in top_qb_prospects.items():
    print(f"first round QBs in {year}: {prospects}")

first round QBs in 2010: ['Sam Bradford', 'Tim Tebow']
first round QBs in 2011: ['Cam Newton', 'Jake Locker', 'Blaine Gabbert', 'Christian Ponder']
first round QBs in 2012: ['Andrew Luck', 'Robert Griffin III', 'Ryan Tannehill', 'Brandon Weeden']
first round QBs in 2013: ['EJ Manuel']
first round QBs in 2014: ['Blake Bortles', 'Johnny Manziel', 'Teddy Bridgewater']
first round QBs in 2015: ['Jameis Winston', 'Marcus Mariota']
first round QBs in 2016: ['Jared Goff', 'Carson Wentz', 'Paxton Lynch']
first round QBs in 2017: ['Mitchell Trubisky', 'Patrick Mahomes', 'Deshaun Watson']
first round QBs in 2018: ['Baker Mayfield', 'Sam Darnold', 'Josh Allen', 'Josh Rosen', 'Lamar Jackson']
first round QBs in 2019: ['Kyler Murray', 'Daniel Jones', 'Dwayne Haskins']
first round QBs in 2020: ['Joe Burrow', 'Tua Tagovailoa', 'Justin Herbert', 'Jordan Love']
first round QBs in 2021: ['Trevor Lawrence', 'Zach Wilson', 'Trey Lance', 'Justin Fields', 'Mac Jones']
first round QBs in 2022: ['Kenny Picket

## Hypothesis Test?

| Anatomy of the hypothesis test |  Answer  |
|:------------------------------:|:--------:|
| Assumption                     | ... |
| Null hypothesis                | ... |
| Alternate hypothesis           | ... |
| Test statistic                 | ... |
| Rejection region shape         | ... |